In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prerequisites

In [6]:
!git clone https://github.com/anna-marshalova/hotel-booking-assistant

Cloning into 'hotel-booking-assistant'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 178 (delta 79), reused 143 (delta 44), pack-reused 0
Receiving objects: 100% (178/178), 56.19 KiB | 6.24 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
%cd hotel-booking-assistant

In [8]:
!pip -q install -r requirements.txt

In [ ]:
import os

os.environ["HF_TOKEN"] = ""
os.environ["HF_USERNAME"] = ""

In [6]:
from src.model.pipeline import TextGenerationPipeline

pipe = TextGenerationPipeline()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-02 12:01:48.049284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 12:01:48.049401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 12:01:48.201239: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Train

In [ ]:
from src.dataset.dataset_loading import get_train_dataset
from src.model.training import get_trainer

In [ ]:
train_dataset = get_train_dataset(pipe.tokenizer)

In [ ]:
!pip -q install wandb

In [ ]:
import os

import wandb

wandb.init(project="llm-assistant-training")
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [ ]:
trainer = get_trainer(pipe.model, pipe.tokenizer, train_dataset)

In [ ]:
trainer.train()

# Inference

In [ ]:
from src.assistant.assistant import Assistant

In [ ]:
from src.paths import HOTEL_DATA_PATH

hotel_assistant = Assistant(pipe, HOTEL_DATA_PATH / "test_hotels.json")
hotel_assistant.run()

# Evaluate

In [11]:
from src.dataset.dataset_loading import get_test_datasets

In [12]:
slot_test_dataset, bot_test_dataset = get_test_datasets()

Generating train split:   0%|          | 0/9254 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1720 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1720 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1720 [00:00<?, ? examples/s]

In [14]:
from src.evaluation.slot_evaluation import compute_slot_metrics, eval_slot_filling

true_slots, pred_slots, results = eval_slot_filling(
    slot_test_dataset, pipe, batch_size=1
)
slot_filling_metrics_df = compute_slot_metrics(true_slots, pred_slots)

  0%|          | 0/860 [00:00<?, ?it/s]

In [ ]:
!git clone https://github.com/maszhongming/UniEval.git
%cd UniEval
!pip -q install -r requirements.txt
%cd ..

In [32]:
from src.evaluation.chat_response_generation import get_chat_responses

test_chat_responses = get_chat_responses(bot_test_dataset, pipe, batch_size=1)

  0%|          | 0/860 [00:00<?, ?it/s]

In [81]:
from src.evaluation.chat_eval import evaluate_chats

chat_eval_df = evaluate_chats(test_chat_responses)

  0%|          | 0/860 [00:00<?, ?it/s]

Evaluating naturalness of 256 samples !!!


100%|██████████| 32/32 [00:32<00:00,  1.02s/it]


Evaluating coherence of 256 samples !!!


100%|██████████| 32/32 [00:33<00:00,  1.05s/it]


...

100%|██████████| 157/157 [05:13<00:00,  1.99s/it]


Evaluating groundedness of 92 samples !!!


100%|██████████| 12/12 [00:34<00:00,  2.87s/it]


Evaluating understandability of 92 samples !!!


100%|██████████| 12/12 [00:11<00:00,  1.01it/s]
